# How is power for different number of effects?

Since simulated effects have certain correlation, we provide `mean_corX` to quantify such correlation. For each fixed number of nonzero effects $T \in \{2,3,5,10,20\}$, we investigate SuSiE power as pve changes. 

## Results

We observe that in the GTEx data: 

* As the number of effects increases, it becomes more difficult to achieve high power even if pve is large.

* `mean_corX` does not seem to be a factor that causes a change in power. A decline in power is directly related to an increased number of nonzero effects. 

* Furthermore, fdr also becomes larger when we have more number of nonzero effects.

In [29]:
dscout.summary[dscout.summary$effect_num==2,]

effect_num pve  mean_corX power fdr cs_num
2  2          0.01 0.3371353 0.10  0    2    
8  2          0.02 0.3371353 0.15  0    3    
14 2          0.03 0.3371353 0.25  0    5    
20 2          0.05 0.3371353 0.45  0    9    
26 2          0.10 0.3371353 0.45  0    9    
32 2          0.20 0.3371353 0.50  0   10    
38 2          0.40 0.3371353 0.50  0   10    
44 2          0.50 0.3371353 0.60  0   12    
50 2          0.70 0.3371353 0.65  0   13    
56 2          0.90 0.3371353 0.75  0   15    
62 2          0.99 0.3371353 0.90  0   18

In [16]:
dscout.summary[dscout.summary$effect_num==3,]

effect_num pve  mean_corX power  fdr
3  3          0.01 0.153476  0.0000 0  
9  3          0.02 0.153476  0.0667 0  
15 3          0.03 0.153476  0.1333 0  
21 3          0.05 0.153476  0.2333 0  
27 3          0.10 0.153476  0.3333 0  
33 3          0.20 0.153476  0.3333 0  
39 3          0.40 0.153476  0.4333 0  
45 3          0.50 0.153476  0.4667 0  
51 3          0.70 0.153476  0.4667 0  
57 3          0.90 0.153476  0.5000 0  
63 3          0.99 0.153476  0.6000 0

In [17]:
dscout.summary[dscout.summary$effect_num==5,]

effect_num pve  mean_corX  power fdr   
4  5          0.01 0.08486093 0.00  1.0000
10 5          0.02 0.08486093 0.06  0.2500
16 5          0.03 0.08486093 0.08  0.2000
22 5          0.05 0.08486093 0.14  0.0000
28 5          0.10 0.08486093 0.20  0.0000
34 5          0.20 0.08486093 0.22  0.0000
40 5          0.40 0.08486093 0.28  0.0667
46 5          0.50 0.08486093 0.28  0.0667
52 5          0.70 0.08486093 0.36  0.0000
58 5          0.90 0.08486093 0.38  0.0000
64 5          0.99 0.08486093 0.52  0.0000

In [18]:
dscout.summary[dscout.summary$effect_num==10,]

effect_num pve  mean_corX power fdr   
5  10         0.01 0.1450763 0.01  0.0000
11 10         0.02 0.1450763 0.02  0.0000
17 10         0.03 0.1450763 0.04  0.0000
23 10         0.05 0.1450763 0.08  0.1111
29 10         0.10 0.1450763 0.09  0.1000
35 10         0.20 0.1450763 0.12  0.0769
41 10         0.40 0.1450763 0.17  0.0556
47 10         0.50 0.1450763 0.17  0.0556
53 10         0.70 0.1450763 0.20  0.0476
59 10         0.90 0.1605803 0.29  0.0645
65 10         0.99 0.1450763 0.40  0.0698

In [19]:
dscout.summary[dscout.summary$effect_num==20,]

effect_num pve  mean_corX power fdr   
6  20         0.01 0.1680549 0.000 0.0000
12 20         0.02 0.1760362 0.005 0.5000
18 20         0.03 0.1760362 0.015 0.2500
24 20         0.05 0.1760362 0.030 0.0000
30 20         0.10 0.1760362 0.045 0.0000
36 20         0.20 0.1760362 0.075 0.1176
42 20         0.40 0.1760362 0.100 0.0909
48 20         0.50 0.1760362 0.105 0.1250
54 20         0.70 0.1760362 0.125 0.1667
60 20         0.90 0.1760362 0.180 0.1429
66 20         0.99 0.1760362 0.340 0.0423

## Code details

In [24]:
dscout_Q1 = readRDS('dscout_Q1.rds')
dscout_Q1 = dscout_Q1[!is.na(dscout_Q1$sim_gaussian.output.file),]
dscout_Q1 = dscout_Q1[!is.na(dscout_Q1$susie.output.file),]

In [25]:
dscout_df = data.frame(dscout_Q1$sim_gaussian.effect_num, dscout_Q1$sim_gaussian.pve, 
                       dscout_Q1$sim_gaussian.mean_corX,dscout_Q1$score.hit, dscout_Q1$score.signal_num)
names(dscout_df) = c('effect_num', 'pve', 'mean_corX', 'hit', 'cs_num')

In [26]:
corX.summary = aggregate(mean_corX ~ effect_num + pve, dscout_df, mean)

In [27]:
dscout.summary = aggregate(hit ~ effect_num + pve, dscout_df, sum)
dscout.summary$mean_corX = corX.summary$mean_corX
dscout.summary$power = round(dscout.summary$hit / (dscout.summary$effect_num*10),4)
fdr.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, sum)
fdr.summary$fdr = round(1 - dscout.summary$hit / fdr.summary$cs_num, 4)
dscout.summary$hit = NULL
dscout.summary$fdr = fdr.summary$fdr
dscout.summary$cs_num = fdr.summary$cs_num

In [28]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0